# Steam Predicter

In [1]:
import os
import re
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn import metrics
# from sklearn.linear_model import LogisticRegression
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.pipeline import FeatureUnion
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.pipeline import Pipeline

# Cleaning datasets




In [2]:
DATASET_PATH = os.path.join("datasets")

In [3]:
def load_data(filename, path):
    csv_path = os.path.join(path, filename)
    return pd.read_csv(csv_path)

In [4]:
steam_data = load_data("steam_data.csv", DATASET_PATH)
text_data = load_data("text_content.csv", DATASET_PATH)
dataset = load_data("steam.csv", DATASET_PATH)
dataset2 = load_data("steamspy_tag_data.csv", DATASET_PATH)
dataset3 = load_data("steam_support_info.csv", DATASET_PATH)
dataset4 = load_data("steam_requirements_data.csv", DATASET_PATH)
dataset5 = load_data("steam_media_data.csv", DATASET_PATH)
dataset6 = load_data("steam_description_data.csv", DATASET_PATH)

# Datasets
Let's take a look at datasets. We want to predict the overall user score based on `all_reviews` column found in `steam_data.csv`. However, we don't have enough useful data in this data set, so we will take a look at other datasets:

* `text_content.csv` contains games descriptions and popular tags given by the community.
* `steam.csv` contains a lot of useful data such as genres, playtime and number of owners.
* `steam_description_data.csv` contains a much more detailed description of games.

We would link datasets through appid - unique identifier of every game available on Steam.


In [5]:
steam_data.head()

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,https://store.steampowered.com/app/945360/Amon...,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...","Overwhelmingly Positive(224,878)- 95% of the 2...","Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,Steam AchievementsFull controller supportSteam...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Very Positive(90,780)- 88% of the 90,780 user ...","Very Positive(4,843,904)- 87% of the 4,843,904...","Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,Play Counter-Strike: Global OffensiveFree to P...,-,-
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,MMOOnline PvPOnline Co-opSteam AchievementsFul...,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Very Positive(32,436)- 84% of the 32,436 user ...","Very Positive(223,706)- 80% of the 223,706 use...","Aug 3, 2020",Mediatonic,Devolver Digital,Buy Fall Guys$19.99Add to Cart,-,-
3,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,Single-playerOnline PvPSteam AchievementsSteam...,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Very Positive(5,359)- 91% of the 5,359 user re...","Very Positive(18,951)- 92% of the 18,951 user ...","Sep 1, 2020",Paradox Development Studio,Paradox Interactive,Buy Crusader Kings III$49.99Add to Cart,BloodLanguagePartial NuditySexual ThemesUse of...,https://steamstore-a.akamaihd.net/public/share...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2,Single-playerOnline PvPOnline Co-opSteam Achie...,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Very Positive(9,147)- 87% of the 9,147 user re...","Very Positive(284,689)- 86% of the 284,689 use...","Oct 1, 2019",Bungie,Bungie,Play Destiny 2Free To PlayPlay Game,Blood Language Violence,https://steamstore-a.akamaihd.net/public/share...


In [6]:
text_data.head()

,url,desc,full_desc,requirements,popu_tags
0,https://store.steampowered.com/app/945360/Amon...,An online and local party game of teamwork and...,About This GamePlay with 4-10 player online or...,System RequirementsMinimum:OS: Windows 7 SP1+P...,MultiplayerOnline Co-OpSpaceSocial DeductionSu...
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive (CS: GO) expa...,About This GameCounter-Strike: Global Offensiv...,System RequirementsWindowsMac OS XSteamOS + Li...,FPSShooterMultiplayerCompetitiveActionTeam-Bas...
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys is a massively multiplayer party gam...,About This GameFall Guys: Ultimate Knockout fl...,System RequirementsMinimum:Requires a 64-bit p...,MultiplayerFunnyBattle RoyaleOnline Co-OpFamil...
3,https://store.steampowered.com/app/1158310/Cru...,Paradox Development Studio brings you the sequ...,About This GameParadox Development Studio brin...,System RequirementsWindowsMac OS XSteamOS + Li...,StrategyRPGSimulationMedievalGrand StrategyHis...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2 is an action MMO with a single evolv...,About This GameDive into the world of Destiny ...,System RequirementsMinimum:Requires a 64-bit p...,Free to PlayLooter ShooterMultiplayerFPSAction...


In [7]:
dataset.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [8]:
dataset2.head()

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550
1,20,0,71,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,122,0,0,0
3,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,77,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
dataset3.head()

,steam_appid,website,support_url,support_email
0,10,NaN,http://steamcommunity.com/app/10,NaN
1,30,http://www.dayofdefeat.com/,NaN,NaN
2,50,NaN,https://help.steampowered.com,NaN
3,70,http://www.half-life.com/,http://steamcommunity.com/app/70,NaN
4,80,NaN,http://steamcommunity.com/app/80,NaN


In [10]:
dataset4.head()

,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [11]:
dataset5.head()

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN
1,20,https://steamcdn-a.akamaihd.net/steam/apps/20/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN
2,30,https://steamcdn-a.akamaihd.net/steam/apps/30/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/30/...,NaN
3,40,https://steamcdn-a.akamaihd.net/steam/apps/40/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN
4,50,https://steamcdn-a.akamaihd.net/steam/apps/50/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN


In [12]:
dataset6.head()

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


## Cleaning data

Extract appid from url and delete all items that aren't games (like bundles).

In [13]:
# https://store.steampowered.com/app/945360/Among_Us/?snr=1_7_7_230_150_1  <- how url look like
def clean_url(url):
    url_left = url.split("app/")
    if len(url_left) < 2:
        return np.NaN # return NaN if it isn't the game
    url_left = url_left[1].split("/")

    return url_left[0]

def clean_dataset_url(dataset):
    dataset.url = dataset.url.apply(lambda x : clean_url(x))

    dataset = dataset.drop(dataset[dataset.url == np.NaN].index)

    return dataset


In [14]:
steam_data = clean_dataset_url(steam_data)
steam_data.head()

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,945360,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...","Overwhelmingly Positive(224,878)- 95% of the 2...","Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-
1,730,Counter-Strike: Global Offensive,Steam AchievementsFull controller supportSteam...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Very Positive(90,780)- 88% of the 90,780 user ...","Very Positive(4,843,904)- 87% of the 4,843,904...","Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,Play Counter-Strike: Global OffensiveFree to P...,-,-
2,1097150,Fall Guys: Ultimate Knockout,MMOOnline PvPOnline Co-opSteam AchievementsFul...,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Very Positive(32,436)- 84% of the 32,436 user ...","Very Positive(223,706)- 80% of the 223,706 use...","Aug 3, 2020",Mediatonic,Devolver Digital,Buy Fall Guys$19.99Add to Cart,-,-
3,1158310,Crusader Kings III,Single-playerOnline PvPSteam AchievementsSteam...,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Very Positive(5,359)- 91% of the 5,359 user re...","Very Positive(18,951)- 92% of the 18,951 user ...","Sep 1, 2020",Paradox Development Studio,Paradox Interactive,Buy Crusader Kings III$49.99Add to Cart,BloodLanguagePartial NuditySexual ThemesUse of...,https://steamstore-a.akamaihd.net/public/share...
4,1085660,Destiny 2,Single-playerOnline PvPOnline Co-opSteam Achie...,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Very Positive(9,147)- 87% of the 9,147 user re...","Very Positive(284,689)- 86% of the 284,689 use...","Oct 1, 2019",Bungie,Bungie,Play Destiny 2Free To PlayPlay Game,Blood Language Violence,https://steamstore-a.akamaihd.net/public/share...


In [15]:
text_data = clean_dataset_url(text_data)
text_data.head()

,url,desc,full_desc,requirements,popu_tags
0,945360,An online and local party game of teamwork and...,About This GamePlay with 4-10 player online or...,System RequirementsMinimum:OS: Windows 7 SP1+P...,MultiplayerOnline Co-OpSpaceSocial DeductionSu...
1,730,Counter-Strike: Global Offensive (CS: GO) expa...,About This GameCounter-Strike: Global Offensiv...,System RequirementsWindowsMac OS XSteamOS + Li...,FPSShooterMultiplayerCompetitiveActionTeam-Bas...
2,1097150,Fall Guys is a massively multiplayer party gam...,About This GameFall Guys: Ultimate Knockout fl...,System RequirementsMinimum:Requires a 64-bit p...,MultiplayerFunnyBattle RoyaleOnline Co-OpFamil...
3,1158310,Paradox Development Studio brings you the sequ...,About This GameParadox Development Studio brin...,System RequirementsWindowsMac OS XSteamOS + Li...,StrategyRPGSimulationMedievalGrand StrategyHis...
4,1085660,Destiny 2 is an action MMO with a single evolv...,About This GameDive into the world of Destiny ...,System RequirementsMinimum:Requires a 64-bit p...,Free to PlayLooter ShooterMultiplayerFPSAction...


Extract % of positive reviews from `all_reviews` column.

In [16]:
# Very Positive(4,843,904)- 87% of the 4,843,904 user reviews for this game are positive. <- sample
def clean_reviews(reviews_string):
    if type(reviews_string) is not str:
        return np.NaN
        
    m = re.search("- (.+?)%", reviews_string)
    if m:
        return int(m.group(1))
    else:
        return np.NaN

In [17]:
steam_data.all_reviews = steam_data.all_reviews.apply(lambda x : clean_reviews(x))

dataset = steam_data.drop(steam_data[steam_data.url == np.NaN].index)

steam_data.head()

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,945360,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...",95.0,"Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-
1,730,Counter-Strike: Global Offensive,Steam AchievementsFull controller supportSteam...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Very Positive(90,780)- 88% of the 90,780 user ...",87.0,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,Play Counter-Strike: Global OffensiveFree to P...,-,-
2,1097150,Fall Guys: Ultimate Knockout,MMOOnline PvPOnline Co-opSteam AchievementsFul...,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Very Positive(32,436)- 84% of the 32,436 user ...",80.0,"Aug 3, 2020",Mediatonic,Devolver Digital,Buy Fall Guys$19.99Add to Cart,-,-
3,1158310,Crusader Kings III,Single-playerOnline PvPSteam AchievementsSteam...,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Very Positive(5,359)- 91% of the 5,359 user re...",92.0,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,Buy Crusader Kings III$49.99Add to Cart,BloodLanguagePartial NuditySexual ThemesUse of...,https://steamstore-a.akamaihd.net/public/share...
4,1085660,Destiny 2,Single-playerOnline PvPOnline Co-opSteam Achie...,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Very Positive(9,147)- 87% of the 9,147 user re...",86.0,"Oct 1, 2019",Bungie,Bungie,Play Destiny 2Free To PlayPlay Game,Blood Language Violence,https://steamstore-a.akamaihd.net/public/share...


Combine `steam_data` and `text_content` datasets.

In [18]:
x = steam_data.merge(text_data, how="inner", on="url", sort=True)

x = x.drop_duplicates(subset="url", keep="first")
x

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url,desc,full_desc,requirements,popu_tags
0,10,Counter-Strike,Online PvPShared/Split Screen PvPValve Anti-Ch...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"Overwhelmingly Positive(1,186)- 97% of the 1,1...",96.0,"Nov 1, 2000",Valve,Valve,Buy Counter-Strike: Condition ZeroIncludes 2 i...,-,-,Play the world's number 1 online action game. ...,About This GamePlay the world's number 1 onlin...,System RequirementsWindowsMac OS XSteamOS + Li...,ActionFPSMultiplayerShooterClassicTeam-BasedFi...
1,1000000,ASCENXION,Single-playerPartial Controller SupportSteam i...,https://steamcdn-a.akamaihd.net/steam/apps/100...,No user reviews,NaN,Winter 2020,IndigoBlue Game Studio,-,Download ASCENXION DEMODownload Demo (Language...,-,-,'ASCENXION' is a game combining shoot 'em up a...,About This Game'ASCENXION' is a game combining...,System RequirementsMinimum:OS: Windows 7 / 8 /...,ActionAdventureIndieShoot 'Em UpMetroidvaniaSh...
10,1000010,Crown Trick,Single-playerSteam AchievementsSteam Trading C...,https://steamcdn-a.akamaihd.net/steam/apps/100...,Very Positive(581)- 80% of the 581 user review...,NaN,"16 Oct, 2020",NEXT Studios,-,Buy Crown TrickSPECIAL PROMOTION! Offer ends 2...,-,-,"Enter a labyrinth that moves as you move, wher...",About This GameCrown Trick is a beautifully an...,System RequirementsMinimum:OS: Windows 7 (32-b...,RoguelikeTurn-Based CombatRPGPerma DeathRoguel...
14,1000030,"Cook, Serve, Delicious! 3?!",Single-playerShared/Split Screen Co-opSteam Ac...,https://steamcdn-a.akamaihd.net/steam/apps/100...,Very Positive(40)- 95% of the 40 user reviews ...,96.0,"Oct 14, 2020",Vertigo Gaming Inc.,Vertigo Gaming Inc.,"Buy Cook, Serve, Delicious! 3?!$19.99Add to Cart",Use of AlcoholFantasy Violence,https://steamstore-a.akamaihd.net/public/share...,"Cook, serve and manage your food truck as you ...",About This GameHit the road in this massive se...,System RequirementsMinimum:OS: Windows 7/8/10...,TypingCasualManagementDifficultArcadeStrategy2...
15,1000040,细胞战争,Single-playerProfile Features Limited,https://steamcdn-a.akamaihd.net/steam/apps/100...,1 user reviews- Need more user reviews to gene...,NaN,"30 Mar, 2019",DoubleC Games,-,Buy 细胞战争NT$ 45Add to Cart,-,-,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,About This Game这款游戏以人体内部为背景，作为侵入人体的细菌，你需要从一个器官...,System RequirementsMinimum:Requires a 64-bit p...,ActionCasualIndieSimulation+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106582,999890,Bruken,Single-playerProfile Features Limited,https://cdn.cloudflare.steamstatic.com/steam/a...,No user reviews,NaN,"Apr 11, 2019",FlairBot Games,-,Buy Bruken$2.99Add to Cart,-,-,"Explore a large spherical world, enjoying all ...",About This GameBruken? That's a weird name... ...,System RequirementsMinimum:OS: Windows 10Proce...,IndieCasualAdventure+
106583,999930,Fantasy Sino-Japanese War 幻想甲午,Single-playerPartial Controller SupportSteam C...,https://cdn.cloudflare.steamstatic.com/steam/a...,2 user reviews- Need more user reviews to gene...,NaN,"Jan 11, 2019",张八万工作室,-,Download Fantasy Sino-Japanese War 幻想甲午 Demo试玩...,-,-,You unfortunately passed through the Sino-Japa...,About This GameYou unfortunately passed throug...,System RequirementsMinimum:OS: Microsoft® Wind...,Early AccessIndieRPGStrategy+
106587,999940,Gods and Kings：Standard Edition,Single-playerDownloadable ContentSteam CloudPr...,https://cdn.cloudflare.steamstatic.com/steam/a...,1 user reviews- Need more user reviews to gene...,NaN,"Jan 4, 2019",Xuanjin Studio,-,Buy Gods and Kings：Standard Edition$10.99Add t...,-,-,-,"About This Content""Standard Edition"" add many ...",System RequirementsMinimum:OS: Windows XPProce...,StrategyRPGIndie+
106588,999990,Bouncing Hero,Single-playerSteam AchievementsSteam CloudProf...,https://cdn.cloudflare.steamstatic.com/steam/a...,7 user reviews- Need more user reviews to gene...,NaN,"Sep 1, 2019","ZPink, zniq.co",-,Buy Bouncing He

Delete unnecessary columns.